# Plotting with pymovements

Pymovements provides a collection of built-in plotting functions 
to visualize gaze data in both temporal and spatial dimensions.

These functions make it easy to explore and present your data, 
from individual trial visualizations to aggregated participant-level analyses.

In this tutorial, you’ll learn how to:

- load and prepare a sample dataset for plotting
- compute the necessary properties
- plot gaze traces over time (traceplot)
- visualize fixations on a stimulus (scanpathplot)
- generate heatmaps showing gaze density (heatmap)
- plot the saccadic main sequence (main-sequence)
- style the plot

All examples use the small and reproducible {doc}`ToyDataset <../datasets/ToyDataset>` 
that comes with pymovements.

## Loading and preprocessing data

In [ ]:
import pymovements as pm

dataset = pm.Dataset('ToyDataset', path='data/ToyDataset')
dataset.download()
dataset.load()

# convert the raw x and y coordinates in pixels to degrees in visual angle
dataset.pix2deg()

# convert these positions into velocitites
dataset.pos2vel()

Check if all the expected columns are in place

In [ ]:
#  You should see the following columns in gaze.samples:
#  time, stimuli_x, stimuli_y,text_id, page_id, pixel, position, velocity
dataset.gaze[0]

## Detecting events and computing properties

Eye-tracking is are typically segmented into fixations and saccades. Fixations represent moments when the eyes remain relatively still, allowing visual information to be processed, while saccades are the rapid movements between fixations that reposition the gaze. Detecting these events and computing their properties, such as fixation duration, saccade amplitude, and peak velocity, provides the foundation for analyzing visual behavior and understanding how participants explore a stimulus.

### Fixations

We can detect fixations by applying a specific event detection method. 

The **threshold identification algorithm** (I-VT or IVT) separates fixation and saccade points based on their point-to-point velocities. It classifies each point as a fixation if the velocity is below the given threshold. Consecutive fixation points are merged into one fixation. 20 degrees/sec is often set as a default maximum threshold. 
Read more about the [IVT algorithm in the documentation](https://pymovements.readthedocs.io/en/stable/reference/api/pymovements.events.detection.ivt.html).

The **dispersion-threshold identification algorithm** (I-DT or IDT) identifies fixations by grouping consecutive points within a maximum separation (dispersion) threshold and a minimum duration threshold. The algorithm uses a moving window to check the dispersion of the points in the window. If the dispersion is below the threshold, the window represents a fixation, and the window is expanded until the dispersion is above threshold.
Read more about our implementation of the [IDT algorithm](https://pymovements.readthedocs.io/en/stable/reference/api/pymovements.events.detection.idt.html).

In [ ]:
# detect fixations using the I-DT algorithm

# Defaults:
# minimum_duration = 100 ms,
# dispersion_threshold = 1.0 degree
dataset.detect('idt', minimum_duration=100, dispersion_threshold=1.0)

### Saccades

Saccades are rapid eye movements that shift the point of fixation from one location to another. We detect saccades (or micro-saccades) from velocity gaze sequence. The `microsaccades` algorithm has a noise-adaptive velocity threshold parameter, which can be set explicitly.

In [ ]:
# detect saccades using the microsaccades algorithm

# Defaults:
# minimum_duration = 6 ms (or 6 samples if timesteps is None)
# threshold_factor = 5
dataset.detect_events('microsaccades', minimum_duration=6, threshold_factor=5)

#### Compute Properties

In [ ]:
# compute event properties, such as the mean position of each fixation
dataset.compute_event_properties(("location", {"position_column": "pixel"}))

# compute amplitude and peak velocity of the detected saccades
dataset.compute_event_properties(['amplitude', 'peak_velocity'])

#  the DataFrame with detected events should now contain the following columns:
#  text_id, page_id, name, onset, offset, duration, amplitude, peak_velocity, location
dataset.events[0]

## Making the Traceplot

A traceplot visualizes the raw gaze samples. It represents eye movements as a continuous trajectory connecting gaze points, where each point corresponds to a specific position and moment in time. This helps reveal viewing patterns.

Traceplots are useful for:
- Verifying that gaze data have been parsed and aligned correctly.
- Exploring viewing behavior across conditions or participants.
- Identifying artifacts or data quality issues.

A basic traceplot can be created with only a `Gaze` object:

In [ ]:
pm.plotting.traceplot(dataset.gaze[3])

## Creating the Scanpath Plot

A scanpath plot visualizes the sequence of fixations as circles whose sizes represent their durations. In a complete scanpath plot, each fixation has an arrow pointing towards the next fixation in viewing order (this feature is currently not implemented).

In [ ]:
pm.plotting.scanpathplot(dataset.gaze[3])

We can overlay the scanpath plot with the traceplot to relate fixation order and duration to the corresponding time course of gaze shifts.

In [ ]:
pm.plotting.scanpathplot(dataset.gaze[3], add_traceplot=True)

## Heatmap Plotting

The heatmap visualizes the spatial distribution of gaze samples across the experiment screen. Each cell's color value reflects the cumulative time (in seconds) that the gaze samples were recorded at that position.

Let's create a heatmap using the `heatmap` function from the `pymovements` library. We will use the default `gridsize` of 10x10 with interpolation and display the colorbar.

In [ ]:
pm.plotting.heatmap(dataset.gaze[3])

In [ ]:
# you can customize various aspects of the heatmap plot, such as the grid
# size, color map, and labels.

figure = pm.plotting.heatmap(
    gaze=dataset.gaze[5],
    position_column='pixel',
    origin='upper',
    show_cbar=True,
    cbar_label='Time [s]',
    title='Gaze Heatmap with Interpolation On',
    xlabel='X [pix]',
    ylabel='Y [pix]',
    gridsize=[10, 10],
)

To better understand the effect of the `gridsize` parameter on the heatmap, we can turn off the interpolation. By doing this, we can clearly visualize the individual bins used to calculate the heatmap. With interpolation turned off, the heatmap will display the raw bin values rather than a smoothed representation.

In [ ]:
figure = pm.plotting.heatmap(
    dataset.gaze[5],
    position_column='pixel',
    origin='upper',
    show_cbar=True,
    cbar_label='Time [s]',
    title='Gaze Heatmap with Interpolation Off',
    xlabel='X [pix]',
    ylabel='Y [pix]',
    gridsize=[10, 10],
    interpolation='none'
)

Increasing the `gridsize` parameter results in a finer grid and more detailed heatmap representation. With a higher grid size, we divide the plot into smaller bins, which can capture more nuances in the data distribution

In [ ]:
figure = pm.plotting.heatmap(
    dataset.gaze[5],
    position_column='pixel',
    origin='upper',
    show_cbar=True,
    cbar_label='Time [s]',
    title='Gaze Heatmap with Higher Grid Size',
    xlabel='X [pix]',
    ylabel='Y [pix]',
    gridsize=[25, 25]
)

## Plotting the Saccadic Main Sequence

The saccadic main sequence describes the characteristic relationship between a saccade’s amplitude and its peak velocity: larger saccades tend to be faster, following a nonlinear, saturating curve.

It is commonly used to validate saccade detection and assess data quality, since deviations from the expected pattern can indicate recording errors or atypical oculomotor behavior.

In [ ]:
# show the first three event dataframes.
# note that you can adjust the styling of the plot, e.g. setting a low
# alpha value allows you to see overlapping data points
for event_df in dataset.events[:3]:
    pm.plotting.main_sequence_plot(
        event_df,
        title='Main sequence plot for '
        f'text {event_df[0, "text_id"]}, '
        f'page {event_df[0, "page_id"]}',
        alpha=0.5,
        color='green',
        marker='x',
        marker_size=30,
    )